# Research Notebook
## Kelly Hayes
## Due Date: March 10, 2025

# 1: Experience
## Describe at least one research activity you worked on this week. 

- Reviewed the Adhikari et al. semi-analytical model in the Diemer COLOSSUS module "splashback.py" [1]
- Tried to understand how their model works

## Motivation:

We want to reproduce this semi-analytical model for cold dark matter so we can make our own generalized semi-analytical model that works for other dark matter models. This brings us towards our goal of computing the splashback radius for fuzzy cold dark matter. 

<span style="color: red;"> Slightly more detail about the goal would be nice. By the third journal the overall motivation should become clearer. </span>

# 2: What? (What happened?)
## Describe what happened during your activities for the week.

I first examined the Adhikari et al. model, then I annotated it to try to get a better grasp on what is happening here.

In [4]:
#This first part just imports functions
import numpy as np
import scipy.interpolate
from collections import OrderedDict
import warnings

from colossus import defaults
from colossus.utils import utilities
from colossus.cosmology import cosmology
from colossus.lss import peaks
from colossus.halo import mass_so
from colossus.halo import mass_defs
from colossus.halo import mass_adv
from colossus.halo import profile_nfw

#This part defines a class. In object-oriented programming, a class stores the collective properties we can expect an object to have if it is a member of that class.
#This specifically defines the class of splashback models that COLOSSUS handles
class SplashbackModel():
	"""
	Characteristics of splashback models.
	
	This object contains certain characteristics of a model, most importantly the input quantities
	``q_in`` and output quantities ``q_out`` the model is capable of processing. Additionally, the 
	``depends_on`` field lists the quantities the model depends on. If, for example, z is 
	among these quantities, then the redshift needs to be passed to the :func:`splashbackModel`
	function.
	
	This object does not contain a function pointer to the model functions because those functions
	do not work in a uniform way, necessitating a somewhat more complex decision tree when 
	evaluating them.

	The :data:`models` dictionary contains one item of this class for each available model.
	"""
	def __init__(self):
		
		self.q_in = []
		"""
		The model can be evaluated as a function of these quantities. Valid entries are 
		``Gamma``, ``z``, and ``nu200m``.
		"""
		self.q_out = []
		"""
		The quantities the model can predict. See table above.
		"""
		self.depends_on = []
		"""
		The quantities the model depends on (which need to be passed to the evaluating function). 
		Valid entries are ``Gamma``, ``z``, ``nu``, and ``rspdef``.
		"""
		self.min_Gamma = -np.inf
		"""
		The minimum mass accretion rate where the model is valid.
		"""
		self.max_Gamma = np.inf
		"""
		The maximum mass accretion rate where the model is valid.
		"""
		self.min_nu200m = 0.0
		"""
		The minimum peak height where the model is valid.
		"""
		self.max_nu200m = np.inf
		"""
		The maximum peak height where the model is valid.
		"""
		self.min_z = 0.0
		"""
		The minimum redshift where the model is valid.
		"""
		self.max_z = np.inf
		"""
		The maximum redshift where the model is valid.
		"""

		self.label = ''
		self.style = {}
		
		return
    
#The next part of the code creates an ordered dictionary of models, and assigns them properties
#I'll only include a snippet, since the other parts are the same concepts applied to different models
models = OrderedDict()
"""
Dictionary containing a list of models.

An ordered dictionary containing one :class:`SplashbackModel` entry for each model.
"""

models['adhikari14'] = SplashbackModel()  #this line tells us that adhikari14 is a member of the splashback models class!
#These next lines assign the properties that splashback models have, as defined above in the class definition
models['adhikari14'].q_in = ['Gamma']  
models['adhikari14'].q_out = ['RspR200m', 'MspM200m', 'Deltasp']
models['adhikari14'].depends_on = ['Gamma', 'z']
models['adhikari14'].min_Gamma = 0.2
models['adhikari14'].max_Gamma = 5.9

#The next part is a very long and complicated function. it takes in the properties we just assigned, and outputs different things depending on the model we input!
def splashbackModel(q_out, Gamma = None, nu200m = None, z = None,
				model = defaults.HALO_SPLASHBACK_MODEL,
				statistic = defaults.HALO_SPLASHBACK_STATISTIC,
				rspdef = None):
	"""
	The splashback radius, mass, and scatter.
	
	Depending on the model, this function can return quantities such as :math:`R_{sp} / R_{\\rm 200m}`,
	:math:`M_{\\rm sp} / M_{\\rm 200m}`, the splashback overdensity, or the scatter in these 
	quantities (see the table of models above). The primary input variable can be mass or accretion 
	rate, and other quantities such as redshift may be required depending on which model is chosen
	(see the properties listed in :data:`models` for details).
	
	The function returns only valid predictions for the desired quantity. If any of the parameters
	lie outside the range of validity of the chosen model, no output is returned for that input,
	as indicated by the returned boolean mask. No warning is output in such cases unless there is
	no valid output for any input.
	
	Parameters
	----------
	q_out: str
		Identifier of the output quantity (see listing above). 
	Gamma: array_like
		Mass accretion rate; can be a number or a numpy array. This quantity only needs to be 
		passed for models that depend on the mass accretion rate.
	nu200m: array_like
		The peak height as computed from :math:`M_{\\rm 200m}`; can be a number or a numpy array. This
		quantity only needs to be passed for models that depend on peak height.
	z: array_like
		Redshift; can be a number or a numpy array. This quantity only needs to be passed for 
		models that depend on redshift.
	model: str
		The splashback model to use for the prediction (see table above).
	statistic: str
		Can be ``mean`` or ``median``, determining whether the function returns the best fit to the 
		mean or median profile of a halo sample. This parameter is ignored by most models. Not
		to be mixed up with the definition ``rspdef`` used in the ``diemer20`` model.
	rspdef: str
		The definition of the splashback radius. This parameter is ignored by most models, but 
		used by the ``diemer17`` and ``diemer20`` models to distinguish the ``mean`` of the 
		apocenter distribution or higher percentiles (e.g. ``percentile75``). The function also 
		accepts the newer notation used in the SPARTA code, namely ``sp-apr-mn`` for the mean and 
		``sp-apr-p75`` and so on for percentiles. For models that use this parameter (``diemer17`` 
		and ``diemer20``), it must be given, otherwise the function throws an error. The acceptable
		range of percentiles is between 50 and 90.
	
	Returns
	-------
	y: array_like
		The desired quantity; if the input (``Gamma`` or ``nu200m``) is a number, this is a number, 
		otherwise a numpy array.
	mask: array_like
		A boolean mask of the same dimensions as the input, indicating whether a valid input was 
		returned for each input element (``Gamma`` or ``nu200m``). Note that only the valid values 
		are returned, meaning ``y`` can contain fewer items than ``mask``.
	"""

	# Check that this model exists
	if not model in models:
		raise Exception('Unknown model, %s.' % model)
	m = models[model]

	# Check that this model can perform the requested operation in principle. For this purpose we
	# determine the primary input quantity, either Gamma or nu200m
	if Gamma is None and nu200m is None:
		raise Exception('Either Gamma or nu200m must be passed.')
	if Gamma is not None:
		q_in = 'Gamma'
	else:
		q_in = 'nu200m'
	if not q_in in m.q_in:
		raise Exception('Model %s cannot handle input quantity %s.' % (model, q_in))
	if not q_out in m.q_out:
		raise Exception('Model %s cannot output quantity %s.' % (model, q_out))

	# Check for wrong input of parameters. If multiple fields are given, they must either be 
	# numbers of arrays of the same dimension as the primary input. Create a mask indicating
	# where the results are valid.
	if q_in == 'Gamma':
		if utilities.isArray(Gamma):
			if z is not None and utilities.isArray(z) and len(z) != len(Gamma):
				raise Exception('If z is an array, it must have the same dimensions as Gamma.')
			if nu200m is not None and utilities.isArray(nu200m) and len(nu200m) != len(Gamma):
				raise Exception('If nu200m is an array, it must have the same dimensions as Gamma.')
		elif utilities.isArray(nu200m):
			Gamma = np.ones_like(nu200m) * Gamma
		mask = (Gamma >= m.min_Gamma) & (Gamma <= m.max_Gamma)
		if nu200m is not None:
			mask = mask & (nu200m >= m.min_nu200m) & (nu200m <= m.max_nu200m)
	elif q_in == 'nu200m':
		if utilities.isArray(nu200m):
			if z is not None and utilities.isArray(z) and len(z) != len(nu200m):
				raise Exception('If z is an array, it must have the same dimensions as nu200m.')
		mask = (nu200m >= m.min_nu200m) & (nu200m <= m.max_nu200m)
	else:
		raise Exception('Unknown input quantity, %s.' % q_in)
	if z is not None:
		mask = mask & (z >= m.min_z) & (z <= m.max_z) 
	
	# Now apply the mask to the input array, and return if there are no valid entries.
	if q_in == 'Gamma':
		x = Gamma
	elif q_in == 'nu200m':
		x = nu200m
	x, is_array = utilities.getArray(x)
	x = x.astype(float)
	mask, _ = utilities.getArray(mask)
	x = x[mask]
	if np.count_nonzero(mask) == 0:
		warnings.warn('Found no input values within the limits of model %s.' % model)
		return np.array([]), mask
	if q_in == 'Gamma':
		Gamma = x
		if nu200m is not None and utilities.isArray(nu200m):
			nu200m = nu200m[mask]
	elif q_in == 'nu200m':
		nu200m = x
	
	# Compute Om from z, but only if z is given. Some models use Om rather than z.
	if z is None:
		Om = None
	else:
		cosmo = cosmology.getCurrent()
		Om = cosmo.Om(z)
	ret = None

	if model == 'adhikari14':
		
		Delta, c = modelAdhikari14Deltasp(Gamma, Om)
		if q_out == 'Deltasp':
			ret = Delta
		elif q_out == 'RspR200m':
			ret, _ = modelAdhikari14RspR200m(Delta, c, z)
		elif q_out == 'MspM200m':
			_, ret = modelAdhikari14RspR200m(Delta, c, z)
			
	elif model == 'more15':

		if q_out == 'RspR200m':
			ret = modelMore15RspR200m(z = z, Gamma = Gamma, nu200m = nu200m, statistic = statistic)
		elif q_out == 'MspM200m':
			ret = modelMore15MspM200m(z = z, Gamma = Gamma, nu200m = nu200m, statistic = statistic)
		elif q_out == 'Deltasp':
			msp200m = modelMore15MspM200m(z = z, Gamma = Gamma, nu200m = nu200m, statistic = statistic)
			rsp200m = modelMore15RspR200m(z = z, Gamma = Gamma, nu200m = nu200m, statistic = statistic)
			ret = 200.0 * msp200m / rsp200m**3

	elif model == 'shi16':
		
		if q_out == 'RspR200m':
			ret = modelShi16RspR200m(Gamma, Om)
		elif q_out == 'MspM200m':
			delta = modelShi16Delta(Gamma, Om)
			rspr200m = modelShi16RspR200m(Gamma, Om)
			ret = delta / 200.0 * rspr200m**3
		elif q_out == 'Deltasp':
			ret = modelShi16Delta(Gamma, Om)
	
	elif model == 'mansfield17':
		
		if z < 2.0:
			mask_new = (x <= 5.0)
			mask[mask] = mask_new
			x = x[mask_new]
			if utilities.isArray(nu200m):
				nu200m = nu200m[mask_new]
			if utilities.isArray(Om):
				Om = Om[mask_new]
			
		if q_out == 'RspR200m':
			ret = modelMansfield17RspR200m(x, Om, nu200m)
		elif q_out == 'MspM200m':
			ret = modelMansfield17MspM200m(x, Om)
		elif q_out == 'Deltasp':
			rspr200m = modelMansfield17RspR200m(x, Om, nu200m)
			mspm200m = modelMansfield17MspM200m(x, Om)
			ret = mspm200m * 200.0 / rspr200m**3
		elif q_out == 'RspR200m-1s':
			ret = np.ones((len(x)), float) * 0.046
		elif q_out == 'MspM200m-1s':
			ret = np.ones((len(x)), float) * 0.054
	
	elif model in ['diemer17', 'diemer20']:

		# Check that a definition was given.
		if rspdef is None:
			raise Exception('Need Rsp/Msp definition such as sp-apr-mn or sp-apr-p75, none given.')
		
		# The model is only valid between the 50th and 90th percentile
		p = _modelDiemerPercentileValue(rspdef)
		if (p > 0.0 and p < 0.5) or p > 0.901:
			mask[:] = False
			ret = np.array([])
			return ret, mask
		
		# If only nu200m is given, we compute Gamma from nu and z.
		if q_in == 'nu200m':
			pars = _modelDiemerGetPars(model, 'Gamma', None)
			Gamma = modelDiemerGamma(nu200m, z, pars)
		
		if q_out in ['RspR200m', 'MspM200m']:
			pars = _modelDiemerGetPars(model, q_out, rspdef)
			ret = modelDiemerRspMsp(Gamma, nu200m, z, rspdef, pars)
		
		elif q_out == 'Deltasp':
			pars = _modelDiemerGetPars(model, 'RspR200m', rspdef)
			rsp200m = modelDiemerRspMsp(Gamma, nu200m, z, rspdef, pars)
			pars = _modelDiemerGetPars(model, 'MspM200m', rspdef)
			msp200m = modelDiemerRspMsp(Gamma, nu200m, z, rspdef, pars)
			ret = 200.0 * msp200m / rsp200m**3
		
		else:
			if q_in == 'nu200m':
				if q_out == 'RspR200m-1s':
					ret = np.ones((len(mask)), float) * 0.07
				elif q_out == 'MspM200m-1s':
					ret = np.ones((len(mask)), float) * 0.07
				elif q_out == 'Deltasp-1s':
					ret = np.ones((len(mask)), float) * 0.15
				else:
					raise Exception('Unknown quantity, %s.' % (q_out))
			else:
				if q_out in ['RspR200m-1s', 'MspM200m-1s']:
					pars = _modelDiemerGetPars(model, q_out, rspdef)
					ret = modelDiemerScatter(Gamma, nu200m, rspdef, pars)
				elif q_out == 'Deltasp-1s':
					pars = _modelDiemerGetPars(model, 'RspR200m-1s', rspdef)
					rsp_1s = modelDiemerScatter(Gamma, nu200m, rspdef, pars)
					pars = _modelDiemerGetPars(model, 'MspM200m-1s', rspdef)
					msp_1s = modelDiemerScatter(Gamma, nu200m, rspdef, pars)
					ret = np.sqrt(msp_1s**2 + 3.0 * rsp_1s**2)
				else:
					raise Exception('Unknown quantity, %s.' % (q_out))

				min_scatter = 0.02
				if utilities.isArray(ret):
					ret[ret < min_scatter] = min_scatter
				else:
					ret = max(ret, min_scatter)
	
	else:
		raise Exception('Unknown model, %s.' % model)

	if not is_array:
		ret = ret[0]
		mask = mask[0]

	return ret, mask

#To make this code even more complicated, this next function "wraps" the one above
def splashbackRadius(z, mdef, R = None, M = None, c = None, Gamma = None,
		model = defaults.HALO_SPLASHBACK_MODEL, 
		statistic = defaults.HALO_SPLASHBACK_STATISTIC,
		rspdef = None,
		c_model = defaults.HALO_CONCENTRATION_MODEL,
		profile = defaults.HALO_MASS_CONVERSION_PROFILE):
	"""
	:math:`R_{\\rm sp}` and :math:`M_{\\rm sp}` as a function of spherical overdensity radius or mass.
	
	This function represents a convenient wrapper for the :func:`splashbackModel` function, where
	only a spherical overdensity mass or radius needs to be passed. Additionally, the user can 
	pass any of the parameters to the :func:`splashbackModel` function. Note that the redshift
	must be a number, not an array.
	
	Parameters
	----------
	z: float
		Redshift
	mdef: str
		Mass definition in which any combination of ``R``, ``M``, and ``c`` is given. See 
		:doc:`halo_mass` for details.
	R: array_like
		Spherical overdensity radius in physical :math:`kpc/h`; can be a number or a numpy array.
		Either ``R`` or ``M`` need to be passed, not both.
	M: array_like
		Spherical overdensity mass in :math:`M_{\\odot}/h`; can be a number or a numpy array.
		Either ``R`` or ``M`` need to be passed, not both.
	c: array_like
		Halo concentration; must have the same dimensions as ``R`` or ``M``, or be ``None`` in 
		which case the concentration is computed automatically.
	Gamma: array_like
		The mass accretion rate that can be optionally passed to the splashback model. If this 
		field is set, the splashback model is evaluated with Gamma as the primary input, otherwise
		peak height is the primary input. If ``Gamma`` is an array, it must have the same 
		dimensions as the input ``R`` or ``M``.
	model: str
		The splashback model to use for the prediction (see table above).
	statistic: str
		Can be ``mean`` or ``median``, determining whether the function returns the best fit to the 
		mean or median profile of a halo sample. This parameter is ignored by most models.
	rspdef: str
		The definition of the splashback radius. This parameter is ignored by most models, but 
		used by the ``diemer17`` model to distinguish the ``mean`` of the apocenter distribution
		or higher percentiles (e.g. ``percentile75``). The function also accepts the newer notation
		used in the SPARTA code, namely ``sp-apr-mn`` for the mean and ``sp-apr-p75`` and so on
		for percentiles.
	c_model: str
		The concentration model used to compute c if it is not passed by the user. See the
		:doc:`halo_concentration` module for details.
	profile: str
		The functional form of the profile assumed in the conversion between mass definitions; 
		can be ``nfw`` or ``dk14``.

	Returns
	-------
	Rsp: array_like
		:math:`R_{\\rm sp}` in physical :math:`kpc/h`
	Msp: array_like
		:math:`M_{\\rm sp}` in :math:`M_{\\odot}/h`
	mask: array_like
		A boolean mask of the same dimensions as ``R`` or ``M``, indicating whether a valid input 
		was returned for each input. Note that only the valid outputs are returned, meaning Rsp 
		and Msp can have fewer items than mask.
	"""
	
	if R is None and M is None:
		raise Exception('Either R or M must be passed.')
	if R is not None and M is not None:
		raise Exception('Only R or M can be passed, not both.')
	
	if R is not None:
		R, is_array = utilities.getArray(R)
		R = R.astype(float)
	else:
		M, is_array = utilities.getArray(M)
		M = M.astype(float)
	
	if mdef == '200m':
		if R is None:
			M200m = M
			R200m = mass_so.M_to_R(M200m, z, '200m')
		else:
			R200m = R
			M200m = mass_so.R_to_M(R200m, z, '200m')
	else:
		if M is None:
			M = mass_so.R_to_M(R, z, mdef)
		if c is None:
			M200m, R200m, _ = mass_adv.changeMassDefinitionCModel(M, z, mdef, '200m', 
										profile = profile, c_model = c_model)
		else:
			M200m, R200m, _ = mass_defs.changeMassDefinition(M, c, z, mdef, '200m', 
										profile = profile)
	
	# Final parameter check: if Gamma is given, it must have the same dimensions as R/M.
	if Gamma is not None:
		gamma_is_array = utilities.isArray(Gamma)
		if gamma_is_array:
			if not is_array:
				raise Exception('Gamma is an array, but the given R/M is not. They must agree in dimensions.')
			if len(Gamma) != len(M200m):
				raise Exception('The Gamma array has length %d, the R/M array %d; they must agree in dimensions.' \
							% (len(Gamma), len(M200m)))

	# Perform the actual computations. We first convert mass to peak height and then evaluate the
	# splashback model.			
	nu200m = peaks.peakHeight(M200m, z)
	
	RspR200m, mask1 = splashbackModel('RspR200m', Gamma = Gamma, nu200m = nu200m, z = z, model = model,
				statistic = statistic, rspdef = rspdef)
	MspM200m, mask2 = splashbackModel('MspM200m', Gamma = Gamma, nu200m = nu200m, z = z, model = model,
				statistic = statistic, rspdef = rspdef)
	
	# The masks should be the same but we require both to be true to be safe.
	mask = mask1 & mask2
	
	# Special case: if there is only one element and its mask is False, we need to return some
	# value for R and M.
	if (np.count_nonzero(mask) == 0) and (not is_array):
		Rsp = None
		Msp = None
		mask = False
		
	else:
		Rsp = R200m[mask] * RspR200m
		Msp = M200m[mask] * MspM200m
		if not is_array:
			Rsp = Rsp[0]
			Msp = Msp[0]
			mask = mask[0]
		
	return Rsp, Msp, mask

#the next function is the one that actually does the math for the model we want, 
#however, it doesn't compute splashback radius yet, it's just intermediate steps. we'll need another function for that
def modelAdhikari14Deltasp(s, Om):
	"""
	:math:`\\Delta_{\\rm sp}` and concentration for the Adhikari et al 2014 model.
	
	The model is evaluated numerically, this function returns interpolated results as a function of
	mass accretion rate ``s`` and :math:`\\Omega_{\\rm m}(z)`.
	
	Parameters
	----------
	s: array_like
		Instantaneous mass accretion rate
	Om: array_like
		Matter density of the universe in units of the critical density.

	Returns
	-------
	Delta: array_like
		The overdensity enclosed by the predicted splashback radius, in units of the mean density.
	c: array_like
		The concentration of the predicted NFW profile.
	"""
	
	bins_Om = np.array([1.000e-02, 6.210e-02, 1.142e-01, 1.663e-01, 2.184e-01, 2.705e-01, 3.226e-01, 3.747e-01, 4.268e-01, 4.789e-01, 5.310e-01, 5.831e-01, 6.352e-01, 6.873e-01, 7.394e-01, 7.915e-01, 8.436e-01, 8.957e-01, 9.478e-01, 9.999e-01])
	bins_s = np.array([2.000e-01, 2.392e-01, 2.860e-01, 3.421e-01, 4.091e-01, 4.893e-01, 5.851e-01, 6.998e-01, 8.369e-01, 1.001e+00, 1.197e+00, 1.431e+00, 1.712e+00, 2.047e+00, 2.449e+00, 2.928e+00, 3.502e+00, 4.188e+00, 5.009e+00, 5.990e+00])
	c = np.array([5.503e+02, 2.370e+02, 1.152e+02, 6.164e+01, 3.562e+01, 2.188e+01, 1.410e+01, 9.431e+00, 6.495e+00, 4.571e+00, 3.266e+00, 2.354e+00, 1.701e+00, 1.223e+00, 8.671e-01, 5.971e-01, 3.896e-01, 2.283e-01, 1.014e-01, 8.349e-04])
	Deltasp = np.array([
		[8.131e+02, 8.277e+02, 8.450e+02, 8.673e+02, 8.967e+02, 9.357e+02, 9.860e+02, 1.049e+03, 1.098e+03, 1.191e+03, 1.295e+03, 1.410e+03, 1.532e+03, 1.662e+03, 1.799e+03, 1.947e+03, 2.114e+03, 2.315e+03, 2.573e+03, 2.923e+03],
		[2.342e+02, 2.396e+02, 2.460e+02, 2.540e+02, 2.640e+02, 2.767e+02, 2.924e+02, 3.117e+02, 3.289e+02, 3.567e+02, 3.883e+02, 4.233e+02, 4.612e+02, 5.015e+02, 5.437e+02, 5.875e+02, 6.336e+02, 6.839e+02, 7.418e+02, 8.133e+02],
		[1.633e+02, 1.672e+02, 1.719e+02, 1.777e+02, 1.849e+02, 1.940e+02, 2.051e+02, 2.187e+02, 2.312e+02, 2.507e+02, 2.728e+02, 2.975e+02, 3.243e+02, 3.528e+02, 3.827e+02, 4.136e+02, 4.456e+02, 4.796e+02, 5.173e+02, 5.619e+02],
		[1.328e+02, 1.361e+02, 1.400e+02, 1.448e+02, 1.508e+02, 1.582e+02, 1.674e+02, 1.785e+02, 1.888e+02, 2.047e+02, 2.227e+02, 2.428e+02, 2.647e+02, 2.881e+02, 3.127e+02, 3.380e+02, 3.640e+02, 3.912e+02, 4.207e+02, 4.548e+02],
		[1.152e+02, 1.182e+02, 1.216e+02, 1.258e+02, 1.311e+02, 1.376e+02, 1.455e+02, 1.552e+02, 1.643e+02, 1.780e+02, 1.937e+02, 2.111e+02, 2.302e+02, 2.506e+02, 2.720e+02, 2.941e+02, 3.167e+02, 3.401e+02, 3.651e+02, 3.934e+02],
		[1.036e+02, 1.063e+02, 1.094e+02, 1.132e+02, 1.180e+02, 1.238e+02, 1.310e+02, 1.397e+02, 1.480e+02, 1.603e+02, 1.744e+02, 1.901e+02, 2.073e+02, 2.257e+02, 2.450e+02, 2.648e+02, 2.852e+02, 3.061e+02, 3.282e+02, 3.528e+02],
		[9.521e+01, 9.768e+01, 1.006e+02, 1.042e+02, 1.085e+02, 1.139e+02, 1.205e+02, 1.285e+02, 1.362e+02, 1.475e+02, 1.604e+02, 1.749e+02, 1.907e+02, 2.076e+02, 2.254e+02, 2.437e+02, 2.624e+02, 2.816e+02, 3.016e+02, 3.237e+02],
		[8.884e+01, 9.115e+01, 9.390e+01, 9.723e+01, 1.013e+02, 1.064e+02, 1.126e+02, 1.200e+02, 1.272e+02, 1.377e+02, 1.498e+02, 1.632e+02, 1.780e+02, 1.938e+02, 2.104e+02, 2.276e+02, 2.450e+02, 2.628e+02, 2.814e+02, 3.016e+02],
		[8.378e+01, 8.598e+01, 8.858e+01, 9.174e+01, 9.562e+01, 1.004e+02, 1.062e+02, 1.133e+02, 1.201e+02, 1.300e+02, 1.413e+02, 1.540e+02, 1.680e+02, 1.829e+02, 1.986e+02, 2.148e+02, 2.313e+02, 2.480e+02, 2.653e+02, 2.841e+02],
		[7.967e+01, 8.177e+01, 8.425e+01, 8.727e+01, 9.096e+01, 9.551e+01, 1.011e+02, 1.077e+02, 1.142e+02, 1.237e+02, 1.345e+02, 1.465e+02, 1.597e+02, 1.740e+02, 1.889e+02, 2.043e+02, 2.200e+02, 2.359e+02, 2.523e+02, 2.698e+02],
		[7.623e+01, 7.825e+01, 8.064e+01, 8.353e+01, 8.708e+01, 9.143e+01, 9.674e+01, 1.031e+02, 1.094e+02, 1.184e+02, 1.287e+02, 1.402e+02, 1.529e+02, 1.665e+02, 1.808e+02, 1.956e+02, 2.106e+02, 2.258e+02, 2.414e+02, 2.580e+02],
		[7.331e+01, 7.526e+01, 7.756e+01, 8.035e+01, 8.377e+01, 8.796e+01, 9.307e+01, 9.922e+01, 1.052e+02, 1.139e+02, 1.238e+02, 1.349e+02, 1.471e+02, 1.602e+02, 1.739e+02, 1.881e+02, 2.026e+02, 2.172e+02, 2.321e+02, 2.479e+02],
		[7.080e+01, 7.268e+01, 7.491e+01, 7.761e+01, 8.092e+01, 8.497e+01, 8.990e+01, 9.584e+01, 1.017e+02, 1.100e+02, 1.196e+02, 1.303e+02, 1.420e+02, 1.547e+02, 1.680e+02, 1.817e+02, 1.957e+02, 2.097e+02, 2.241e+02, 2.392e+02],
		[6.860e+01, 7.043e+01, 7.260e+01, 7.522e+01, 7.843e+01, 8.236e+01, 8.714e+01, 9.289e+01, 9.854e+01, 1.066e+02, 1.159e+02, 1.262e+02, 1.376e+02, 1.499e+02, 1.628e+02, 1.761e+02, 1.896e+02, 2.033e+02, 2.171e+02, 2.316e+02],
		[6.666e+01, 6.844e+01, 7.055e+01, 7.310e+01, 7.623e+01, 8.005e+01, 8.469e+01, 9.028e+01, 9.578e+01, 1.036e+02, 1.126e+02, 1.227e+02, 1.337e+02, 1.456e+02, 1.582e+02, 1.711e+02, 1.843e+02, 1.975e+02, 2.109e+02, 2.249e+02],
		[6.493e+01, 6.667e+01, 6.873e+01, 7.122e+01, 7.427e+01, 7.799e+01, 8.252e+01, 8.796e+01, 9.333e+01, 1.010e+02, 1.097e+02, 1.195e+02, 1.303e+02, 1.419e+02, 1.541e+02, 1.667e+02, 1.795e+02, 1.924e+02, 2.054e+02, 2.190e+02],
		[6.338e+01, 6.508e+01, 6.710e+01, 6.953e+01, 7.250e+01, 7.614e+01, 8.056e+01, 8.587e+01, 9.112e+01, 9.858e+01, 1.071e+02, 1.167e+02, 1.272e+02, 1.385e+02, 1.504e+02, 1.628e+02, 1.753e+02, 1.878e+02, 2.005e+02, 2.137e+02],
		[6.197e+01, 6.364e+01, 6.562e+01, 6.800e+01, 7.091e+01, 7.447e+01, 7.879e+01, 8.398e+01, 8.912e+01, 9.642e+01, 1.047e+02, 1.141e+02, 1.243e+02, 1.354e+02, 1.471e+02, 1.592e+02, 1.714e+02, 1.837e+02, 1.961e+02, 2.088e+02],
		[6.070e+01, 6.233e+01, 6.427e+01, 6.661e+01, 6.947e+01, 7.295e+01, 7.719e+01, 8.227e+01, 8.731e+01, 9.444e+01, 1.026e+02, 1.117e+02, 1.218e+02, 1.326e+02, 1.441e+02, 1.559e+02, 1.679e+02, 1.799e+02, 1.920e+02, 2.045e+02],
		[5.953e+01, 6.114e+01, 6.304e+01, 6.534e+01, 6.814e+01, 7.156e+01, 7.572e+01, 8.070e+01, 8.565e+01, 9.264e+01, 1.006e+02, 1.096e+02, 1.194e+02, 1.301e+02, 1.413e+02, 1.529e+02, 1.647e+02, 1.764e+02, 1.883e+02, 2.005e+02]])

	Om_ = np.ones((len(s)), float) * Om
	interp = scipy.interpolate.RectBivariateSpline(bins_Om, bins_s, Deltasp)
	Delta = interp(Om_, s, grid = False)
	
	if np.max(s) > np.max(bins_s):
		raise Exception('Found s = %.2f, greater than max %.2f.' % (np.max(s), np.max(bins_s)))
	if np.min(s) < np.min(bins_s):
		raise Exception('Found s = %.2f, smaller than min %.2f.' % (np.min(s), np.min(bins_s)))
	
	c = np.interp(s, bins_s, c)
	if np.count_nonzero(c < 0.0) > 0:
		raise Exception('Found negative concentration values.')
	
	return Delta, c

#luckily, the next one is shorter, and comes packaged in with it's own description of what's happening

# This function converts the Adhikari et al model overdensity into Rsp. The concentration
# given by the Adhikari refers to R = r_ta / 2 = Rvir, so we create an NFW profile with 
# that concentration. The mass doesn't matter since we're only after the ratio of Rsp and
# R200m, i.e. the model does not depend on the absolute mass scale.

def modelAdhikari14RspR200m(Delta, c, z):
	"""
	:math:`R_{\\rm sp}/R_{\\rm 200m}` and :math:`M_{\\rm sp}/M_{\\rm 200m}` for the Adhikari et al 2014 model.

	Parameters
	----------
	Delta: array_like
		Overdensity; should be computed using the :func:`modelAdhikari14Deltasp` function.
	c: array_like
		Concentration; should be computed using the :func:`modelAdhikari14Deltasp` function.
	z: float
		Redshift	

	Returns
	-------
	RspR200m: array_like
		:math:`R_{\\rm sp}/R_{\\rm 200m}`; has the same dimensions as ``Delta`` and ``c``.
	MspM200m: array_like
		:math:`M_{\\rm sp}/M_{\\rm 200m}`; has the same dimensions as ``Delta`` and ``c``.
	"""

	cosmo = cosmology.getCurrent()
	rho_m = cosmo.rho_m(z)
	rhos, rs = profile_nfw.NFWProfile.nativeParameters(1E10, c, z, 'vir')
	xsp = profile_nfw.NFWProfile.xDelta(rhos, Delta * rho_m)
	Rsp = xsp * rs
	x200m = profile_nfw.NFWProfile.xDelta(rhos, 200.0 * rho_m)
	R200m = x200m * rs
	RspR200m = Rsp / R200m
	MspM200m = Delta * RspR200m**3 / 200.0
	
	return RspR200m, MspM200m

#this marks the end of the code relevant to the model we want

<span style="color: red;"> Code is good but you should show some output. What does the code actually do? </span>

<span style="color: red;"> Is this your code?  </span>

Next, I mark down some of the questions/concerns I had while reviewing this model:
- How does this math work? How much of it do I need to understand?
- Where do equations (1)-(3) in the assigned paper [2] come into play? Do they make an appearance in this code?
- How do I actually run the code above? It's a lot of definining the fundamental functions to the COLOSSUS module, but doesn't give sample parameters so I can actually see it in action. Perhaps I am supposed to use the same method as in the tutorial to call the function, but that doesn't clarify how I can write my own model (which I think was supposed to be the assignment this week)?

# 3: So what? (What does it mean?)
## Describe your results

This code is important, as it is fundamental to the COLOSSUS module. Computing splashback radii, especially with different models, is the bread and butter for how COLOSSUS helps us in our research. Understanding this will help me generalize this semi-analytical model for cold dark matter presented by Adhikari et al. and ultiumately help the group towards our goal to generalize this work for cold and fuzzy dark matter, so we can predict and compute the splashback of that model of dark matter.

<span style="color: red;">  Okay, but we would like to see some results at this stage that you can intepret in terms of the goals of the research project. </span>

# 4. Now what? (What's next?)
## Plan for the next week

I'd like to seek answers for some of the questions I listed above, and try to complete the tasks that I was supposed to complete, before my lack of understanding of how the model was created got in the way. To do this, I will create my own model (?) using equations (1)-(3) in the Adhikari paper to compute splashback radius, and compare this to the results given by the code I annotated above. Hopefully, this model I create will replicate the one above! This means that we can move on to the next step: generalizing this model for other types of dark matter!

<span style="color: red;"> Try making a list of steps that can be achieved in succession to make clearer the way forward. Sometimes small steps are necessary when dealing with large code bases that you are trying to understand.  </span>

# 5. Bibliography

[1] B. Diemer, Bitbucket, https://bitbucket.org/bdiemer/colossus/src/master/colossus/halo/splashback.py (accessed Mar. 10, 2025).
[2] S. Adhikari, N. Dalal, and R. T. Chamberlain, “Splashback in accreting dark matter halos,” Journal of Cosmology and Astroparticle Physics, vol. 2014, no. 11, pp. 019–019, Nov. 2014. doi:10.1088/1475-7516/2014/11/019 

| Category       | Points      |
| ------------- |:------------:|
| Formatting    |       2       |
| Experience    |       2       |
| What?         |       1       |
| So what?      |       2      |
| Now what?     |       1       |
| Bibliography  |       3       |
| Style         |       3       |
| Total         |       14      |